## SING inference  on Google COLAB

by Hyungon Ryu | Sr. Solution Architect at NVIDIA


In this COLAB Jupyter, I'll demonstrate how to generate instrument music sound with SING.

## DevOps

Check Available GPU. COLAB provide Tesla K80.

In [1]:
!nvidia-smi

Thu Dec  6 08:08:09 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    83W / 149W |   1684MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### set max clock

In [2]:
%%bash
#check the environemnt 
echo "Check H/W"
lscpu | grep 'CPU(s):            '
lscpu | grep GHz
echo "memory" && free -m | cut -c-49 |  head -n 2 
echo "storage" && df -h |  cut -c-60 | head -n 2
df -h |  grep '/dev/sda1'
echo " " && nvidia-smi -L | cut -c-17
echo "confure Max Application Clock for K80 875Mhz"
nvidia-smi -ac 2505,875 && nvidia-smi -pm 1
echo " " &&echo "Check S/W"
cat /etc/*-release | grep PRETTY_NAME
python --version 
nvcc --version | grep  tools

Check H/W
CPU(s):              2
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
memory
              total        used        free      
Mem:          13022        3059         163      
storage
Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   13G  328G   4% /
/dev/sda1       365G   16G  350G   5% /opt/bin
 
GPU 0: Tesla K80 
confure Max Application Clock for K80 875Mhz
Applications clocks set to "(MEM 2505, SM 875)" for GPU 00000000:00:04.0
All done.
Persistence mode is already Enabled for GPU 00000000:00:04.0.
All done.
 
Check S/W
PRETTY_NAME="Ubuntu 18.04.1 LTS"
Python 3.6.7
Cuda compilation tools, release 9.2, V9.2.148


### install pytorch 1.0

It will takes 30 seconds to install pytorch 1.0.. 

pytorch 1.0 build number : `torch-nightly-1.0.0.dev20181128`

In [0]:
%%time
%%bash
pip install numpy
pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly-1.0.0.dev20181128-cp36-cp36m-linux_x86_64.whl

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly-1.0.0.dev20181128-cp36-cp36m-linux_x86_64.whl


tcmalloc: large alloc 1073750016 bytes == 0x639e8000 @  0x7f9d814b12a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


CPU times: user 9.26 ms, sys: 5.63 ms, total: 14.9 ms
Wall time: 41.1 s


### install required python utilities
It will takes 30 seconds. 

In [0]:
%%time 
%%bash
pip install -q \
 inflect==0.2.5 \
 librosa==0.6.0 \
 scipy==1.0.0 \
 tensorboardX==1.1 \
 Unidecode==1.0.22 

CPU times: user 2.33 ms, sys: 8.4 ms, total: 10.7 ms
Wall time: 22.7 s


## Git clone SING model

 


In [3]:
%%bash
rm -rf SING
git clone https://github.com/facebookresearch/SING.git

Cloning into 'SING'...


## Download Checkpoint file


### Checpoint file 
official checkpoint from facebook [Link](https://s3.amazonaws.com/sing-models/sing.th)
current link have problem.

In [6]:
%%bash 
wget https://s3.amazonaws.com/sing-models/sing.th

--2018-12-06 08:10:36--  https://s3.amazonaws.com/sing-models/sing.th
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.72.210
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.72.210|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2018-12-06 08:10:36 ERROR 404: Not Found.



### download script

In [0]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)




### Checkpoint file 

SING Model checkpoint file : *`sing.th`* (249MB)

[checkpoint file sing.th(249MB)](https://drive.google.com/file/d/1YeUMecrwiXDsOgIRt5-Zk2rF8Q8pOsoC/view?usp=sharing)

base on default loss and batch_size 2048
- [train_ae][049] 100.0%, loss 0.028624 
- [train_seq][049] 100.0%, loss 0.001227    
- [train_sing][019] 100.0%, loss 0.052460 


In [8]:
%%time
destination="sing.th"
file_id="1YeUMecrwiXDsOgIRt5-Zk2rF8Q8pOsoC"
download_file_from_google_drive(file_id, destination)

CPU times: user 619 ms, sys: 686 ms, total: 1.3 s
Wall time: 4.51 s


In [30]:
%%bash
ls -alh sing.th

-rw-r--r-- 1 root root 244M Dec  6 08:26 sing.th


### copy checkpoint file
default configuration, model localtion is `SING/models/sing.th`

In [0]:
%%bash
mkdir /content/SING/models
cp -rf /content/sing.th /content/SING/models/.

### prepare model


##### SING Model

In [0]:
import sys
sys.path.insert(0, 'SING')

import IPython.display as ipd

from sing import nsynth
from sing.fondation.datasets import RandomSubset
dset = nsynth.get_nsynth_metadata()
train, valid, test = nsynth.make_datasets(dset)


### generate instrument test

In [31]:
evaluation = RandomSubset(test, 2)
open("nsynth_2_test.txt", "w").write("\n".join(
    evaluation[i].metadata['name'] for i in range(len(evaluation))))

59

In [37]:
%%bash
cat  /content/nsynth_2_test.txt

bass_synthetic_126-025-025
synth_lead_synthetic_006-045-050

In [47]:
%%time
%%bash
cd SING
python3 -m sing.generate --cuda --dl /content/nsynth_2_test.txt

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set

CPU times: user 3.6 ms, sys: 20.3 ms, total: 23.9 ms
Wall time: 10.4 s


### Check generated sound

In [49]:
audio_filepath='/content/SING/generated/bass_synthetic_126-025-025.wav'
ipd.Audio(audio_filepath, rate=16000)

In [50]:
audio_filepath='/content/SING/generated/synth_lead_synthetic_006-045-050.wav'
ipd.Audio(audio_filepath, rate=16000)

### Generate more sound

In [51]:
evaluation = RandomSubset(test, 100)
open("nsynth_100_test.txt", "w").write("\n".join(
    evaluation[i].metadata['name'] for i in range(len(evaluation))))

2860

In [52]:
%%time
%%bash
cd SING
python3 -m sing.generate --cuda --dl /content/nsynth_100_test.txt

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set

CPU times: user 6.08 ms, sys: 17.2 ms, total: 23.3 ms
Wall time: 13.8 s


In [53]:
%%bash
head -n 4  /content/nsynth_100_test.txt

bass_synthetic_126-025-025
synth_lead_synthetic_006-045-050
bass_synthetic_065-070-025
keyboard_electronic_063-087-100


In [55]:
audio_filepath='/content/SING/generated/keyboard_electronic_063-087-100.wav'
ipd.Audio(audio_filepath, rate=16000)

In [57]:
audio_filepath='/content/SING/generated/guitar_electronic_004-038-050.wav'
ipd.Audio(audio_filepath, rate=16000)

In [58]:
audio_filepath='/content/SING/generated/organ_electronic_037-047-050.wav'
ipd.Audio(audio_filepath, rate=16000)

In [59]:
audio_filepath='/content/SING/generated/string_acoustic_044-060-025.wav'
ipd.Audio(audio_filepath, rate=16000)

In [60]:
audio_filepath='/content/SING/generated/vocal_acoustic_023-057-127.wav'
ipd.Audio(audio_filepath, rate=16000)